<a href="https://colab.research.google.com/github/ajazturki10/NLP/blob/main/Text_Classification_with_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 7.4 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3128323 sha256=aedd06c60037ce08cafc22be2a87171adee0c4162f55828f502831bdb358d91a
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string

import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import fasttext

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
train_df = pd.read_csv("train.csv")

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df.shape

(7613, 5)

In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocessor(text):
    text = text.lower()
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(r'http\S+', '', text)
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

In [ ]:
train_df["clean_text"] = train_df["text"].apply(preprocessor)

In [ ]:
train_df["label"] = train_df["target"].apply(lambda x: "yes" if x is 1 else "no")

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-10-29a07408fa49>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  train_df["label"] = train_df["target"].apply(lambda x: "yes" if x is 1 else "no")


In [ ]:
train_df.head()

,id,keyword,location,text,target,clean_text,label
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deed are the reason of this earthquake may...,yes
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,yes
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resident asked to 'shelter in place' are b...,yes
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13 000 people receive wildfire evacuation orde...,yes
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska a sm...,yes


In [ ]:
train_df.label.value_counts()

no     4342
yes    3271
Name: label, dtype: int64

In [ ]:
train_df["fasttext_data"] = "__label__" + train_df["label"] + " " + train_df["clean_text"]

In [ ]:
train_df.head()

,id,keyword,location,text,target,clean_text,label,fasttext_data
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deed are the reason of this earthquake may...,yes,__label__yes our deed are the reason of this e...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,yes,__label__yes forest fire near la ronge sask ca...
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resident asked to 'shelter in place' are b...,yes,__label__yes all resident asked to 'shelter in...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13 000 people receive wildfire evacuation orde...,yes,__label__yes 13 000 people receive wildfire ev...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska a sm...,yes,__label__yes just got sent this photo from rub...


In [ ]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df["target"])

In [ ]:
train["fasttext_data"]

6234    __label__yes sassy city girl country hunk stra...
326     __label__no god's kingdom heavenly gov't will ...
997     __label__no mopheme and bigstar johnson are a ...
7269    __label__no vixmeldrew sound like a whirlwind ...
2189    __label__yes malaysia confirms plane debris wa...
                              ...                        
3386    __label__yes ûïa voluntary evacuation is being...
3280    __label__yes rt calestous tanzania elephant po...
305     __label__no pbban temporary 300 russaky89 'arm...
1648    __label__no petition heartless owner that whip...
7569    __label__no marynmck that's beyond adorable i ...
Name: fasttext_data, Length: 6090, dtype: object

In [ ]:
train["fasttext_data"].to_csv("tweets.train", index=False, header=False)
test["fasttext_data"].to_csv("tweets.test", index=False, header=False)

In [ ]:
model = fasttext.train_supervised(input="tweets.train")

In [ ]:
model.test("tweets.test")

(1523, 0.8200919238345371, 0.8200919238345371)

In [ ]:
text = "DTC Fleet Gets 50 New Electric Buses Under FAME India Phase II Scheme"
model.predict(text)

(('__label__no',), array([0.99110091]))

In [ ]:
text = "Strong tremors in Delhi, north India after quake strikes Nepal in dead of the night"
model.predict(text)

(('__label__yes',), array([0.94746894]))